variables:
Peter
Juan
Jim
Jane
Mary
Bruce
Anita

------

Domain:

Name 	Abilities
Peter 	Python and AI
Juan 	Web and AI
Jim 	AI and Systems
Jane 	Python and Database
Mary 	Web and Systems
Bruce 	Systems and Python
Anita 	Web d AI



-----
Constraint:

p(but Ciara know python, so she needs hired 1 more people whos python)ython:2
AI:2
Web:1
Database:1
system:1